In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
import copy
from sklearn.model_selection import train_test_split
import torchvision
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
cdata = torchvision.datasets.EMNIST(root='emnist',split='letters',download=True)


In [ ]:
images = cdata.data.view([124800,1,28,28]).float()
# normalize the images
images /= torch.max(images)
print('\nTensor data:')
print(images.shape)


In [ ]:
letterCategories = cdata.classes[1:]
labels = copy.deepcopy(cdata.targets)-1

In [ ]:
train_data,test_data, train_labels,test_labels = train_test_split(images, labels, test_size=.1)
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)
batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])